In [5]:
import requests
import time
from tqdm import tqdm

SYSTEM_PROMPT = (
    "Ты виртуальный ассистент. Твоя задача - помогать людям с поэзией, отвечая на их запросы."
)
MODEL_NAME = "t-tech/T-lite-it-1.0"

USER_PROMPT_MAIN = '''
Ты — виртуальный ассистент, помогающий людям находить и интерпретировать русскую поэзию.
Ниже приведены отрывки стихотворений, которые могут быть релевантны запросу пользователя. Используй их как источник для вдохновения или цитирования в ответе.
Не выдумывай новых стихов и не приписывай авторам лишнего — работай только с тем, что дано в контексте.
Будь образным, но точным. Если пользователь просит «подборку» — выбери релевантные строки.

Запрос пользователя: {query}
Ответ:
'''

USER_PROMPT_POEM = '''
Ты — поэт-ассистент, вдохновляющийся классической русской поэзией.
На основе запроса пользователя сочини **новое** стихотворение в духе русской поэтической традиции.
Можешь использовать метафоры, эпитеты, ритм, рифму — всё, что придаёт тексту поэтичность.
Если знаешь строки из известных произведений — можешь вдохновляться ими, но **не копируй** дословно и не цитируй.
**НЕ ИСПОЛЬЗУЙ** в стихотворении имена и фамилии авторов.

Запрос пользователя: {query}
Стихотворение:
'''


def generate_sync_vllm(query, is_poem, lora_adapter=None):
    if is_poem:
        prompt = USER_PROMPT_POEM.format(query=query)
    else:
        prompt = USER_PROMPT_MAIN.format(query=query)
    headers = {"Content-Type": "application/json"}
    if lora_adapter:
        headers["x-lora-adapter"] = lora_adapter
    payload = {
        "model": MODEL_NAME,
        "messages": [
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": prompt}
        ],
        "max_tokens": 1024,
        "temperature": 0.7,
        "top_p": 0.9
    }
    try:
        response = requests.post('http://localhost:8000/v1/chat/completions', json=payload, headers=headers, timeout=60)
        response.raise_for_status()
        result = response.json()
        fin_res = result["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"[generate_sync] Ошибка запроса к vLLM: {e}")
        fin_res = ""
    
    if "." in fin_res:
        fin_res = fin_res.rsplit(".", 1)[0] + "."

    return fin_res

VLLM_API_LORA_UPLOAD_URL = 'http://localhost:8000/v1'

def load_lora(lora_name: str, lora_path: str) -> bool:
    try:
        payload = {"lora_name": lora_name, "lora_path": lora_path}
        resp = requests.post(f"{VLLM_API_LORA_UPLOAD_URL}/load_lora_adapter", json=payload, timeout=10)
        return resp.status_code == 200
    except Exception as e:
        print(f"[load_lora] Ошибка: {e}")
        return False


def unload_lora(lora_name: str) -> bool:
    try:
        payload = {"lora_name": lora_name}
        resp = requests.post(f"{VLLM_API_LORA_UPLOAD_URL}/unload_lora_adapter", json=payload, timeout=10)
        return resp.status_code == 200
    except Exception as e:
        print(f"[unload_lora] Ошибка: {e}")
        return False

In [2]:
gen_queries = [
    'Напиши стихотворение в стиле Александра Пушкина. Тема - зима в деревне',
    'Придумай стих в стиле Владимира Маяковского на тему советского патриотизма',
    'Напиши стихотворение на тему осеннего дождливого дня как поэт Осип Мандельштам',
    'Сгенерируй стих на тему безответной любви в стиле Ахматовой',
    'Придумай стихотворение на тему одиночества в городе. Пиши в стиле Иосифа Бродского'
]

In [23]:
lora_name = 'poetry'
lora_path = '/app/data/lora-poetry2'
load_lora(lora_name, lora_path)

True

In [32]:
# gen_response_llm_lst = []
# for q in tqdm(gen_queries):
#     resp = generate_sync_vllm(q, is_poem=True, lora_adapter=lora_name)
#     gen_response_llm_lst.append(resp)

# gen_response_llm_lst2 = []
# for q in tqdm(gen_queries):
#     resp = generate_sync_vllm(q, is_poem=True, lora_adapter=lora_name)
#     gen_response_llm_lst2.append(resp)

gen_response_llm_lst3 = []
for q in tqdm(gen_queries):
    resp = generate_sync_vllm(q, is_poem=True, lora_adapter=lora_name)
    gen_response_llm_lst3.append(resp)

100%|██████████| 5/5 [00:34<00:00,  6.95s/it]


In [29]:
gen_response_llm_lst

['В деревне зимой, где снег лежит,\nКак белый покров на древних полях,\nЛишь ветер шепчет, словно старый брат,\nИ сказки вьюги плетут в тишине.\n\nДома теплом своим обнимают,\nКак материнская ласка в мороз,\nА окна, словно зеркала, блестят,\nОтражая небо в серебряный узор.\n\nСквозь хвойный лес, где сосны стоят\nВо мгле, как стражи зимней ночи,\nПуть к костру ведёт, где огонь тает\nВ снежной пелене, словно светлый сон.\n\nЗима, как волшебница, вдруг явится,\nИ снежинки танцуют в воздухе,\nИ каждый день — как новый рассказ,\nГде сказочный мир в сердце живёт. \n\nВ тишине деревни, где зима царит,\nРаздается пенье соловья,\nИ кажется, что время здесь замирает,\nВ этом мире чудес и мечтаний. \n\nТак в морозном блеске, под лунным светом,\nДуша находит покой и свет,\nИ в сердце зимний ветерок шепчет:\n"Здесь, в деревне, ты нашёл свой дом.',
 'В степях родных, где звонко шепчет ветер,\nГде поднимается над горизонтом солнце,\nСтоит наш город, как страж бесстрашный,\nСимвол силы, что в сердцах 

In [30]:
gen_response_llm_lst2

['В зимней тишине деревни спит,\nПод белым снегом леса молчат.\nИзбушки, как сказочные корабли,\nВ озёрной глади окунулись в мечты.\n\nСеребряный месяц в небе сверкает,\nА звёзды, словно бриллианты, мерцая,\nНад крышами танцуют, светлый луч\nНа сугробы мягкие льют.\n\nХолодный ветер, как шепот старинный,\nПо ветвям шепчет сказания минувших дней.\nЗима – хранительница древних следов,\nГде каждый шаг – как страница веков.\n\nДеревья в снегах, как ледяные боги,\nСтоят, не тронутые временем.\nИ в этом мире, где всё тихо и чисто,\nПросыпается душа, как в сказке чудесной.\n\nТак зимой деревня, словно сон,\nГде время остановилось и замерло.\nИ только сердце в тепле домашнем\nЕщё звучит, как пушкинский стих.',
 'В небе синем, как знамя, взлетает  \nРодины светлый огонь,  \nГде трудом куется победа,  \nГде каждый — как стрела, в ладонях.\n\nМы, как ветер, мчатся впереди,  \nСловно паровоз, что не знает покоя,  \nСквозь земли и воды, сквозь беды и грозы,  \nСлавим Родину, как славят героев.\n\n"

In [33]:
gen_response_llm_lst3

['В деревне зимой, где снега пухом  \nЛежат на полях, как белый покров,  \nТихо дремлет лес, в узорах морозных,  \nИ месяц, как светильник, в небе горит.\n\nДеревья, в снежной бахроме одеты,  \nКак старцы в вуали, стоят вдали.  \nСкрипит под ногами снег хрустальный,  \nИ ветер шепчет: "Зима — царица земли."\n\nВ домах топятся печи, теплом дышат,  \nА в окнах танцуют огни.  \nДетишки играют в снежки,  \nСмеются, веселятся, словно в сказке.\n\nЗима — волшебница, в морозном уборе,  \nСоздает чудеса для всех.  \nИ каждый, кто здесь, в этой тишине,  \nВ душе находит мир и покой.\n\nСквозь ледяные окна виден мир иной,  \nГде сны и мечты плывут, как облака.  \nЗима в деревне — это праздник души,  \nГде каждый найдет свой уголок тепла.',
 'Вперед, к звёздам, как в старину к горизонту!\nСердце бьётся, как паровозный гудок,\nМы — строители нового мира, без страха и трепета,\nГде солнце светит ярче, где мечта живёт в каждом из нас.\n\nКак маяк на берегу, мы — символ силы и чести,\nНаша страна — к

In [35]:
import pandas as pd 

gen_authors_lst = ['Александр Пушкин', 'Владимир Маяковский', 'Осип Мандельштам', 'Анна Ахматова', 'Иосиф Бродский']
gen_themes_lst = ['Зима в деревне', 'Советский патриотизм', 'Осенний дождливый день', 'Безответная любовь', 'Одиночество в городе']

pd.DataFrame({
    'author': gen_authors_lst,
    'theme': gen_themes_lst,
    'queries': gen_queries,
    'LLM_LORA1': gen_response_llm_lst,
    'LLM_LORA2': gen_response_llm_lst2,
    'LLM_LORA3': gen_response_llm_lst3
}).to_csv('../data/ResponsesGen_llm_lora.csv', index=False)

In [36]:
unload_lora(lora_name)

True

In [37]:
lora_name = 'pushkin'
lora_path = '/app/data/lora-poetry-pushkin'
load_lora(lora_name, lora_path)

True

In [42]:
gen_response_llm_lst_p = []
for i in tqdm(range(3)):
    q = gen_queries[0]
    resp = generate_sync_vllm(q, is_poem=True, lora_adapter=lora_name)
    gen_response_llm_lst_p.append(resp)

100%|██████████| 3/3 [00:23<00:00,  7.69s/it]


In [43]:
gen_response_llm_lst_p

['В зимней деревне, где снег лежит,  \nКак белый пух на земле мягкий,  \nТам тишь и мгла, как будто век  \nСпит под покровом морозного ветра.\n\nДеревья в ледяных уборах,  \nСловно хрустальные башни, стоят,  \nИ каждый ствол — как струна в гробах,  \nЧто ждут весны, когда распахнутся врата.\n\nЛуна, как серебряный лепесток,  \nПлывёт сквозь облака, в сонном танце,  \nИ свет её, как древний пророк,  \nВедёт к рассвету, в мир надежды.\n\nСнегири, словно яркие огни,  \nСкользят по веткам, в тишине ночей,  \nИ слышится, как будто вдруг зазвучат  \nСказания старинные в дремоте.\n\nА утром, когда солнце встанет,  \nИ первый луч на крышу упадёт,  \nТо вновь проснётся жизнь в деревне,  \nИ звонкий смех детей раздастся вновь.  \n\nТак зима, как мудрый старец,  \nУкрывает землю в белоснежный покров,  \nЧтоб вновь, как птица из оков,  \nРазбудить её, когда придёт пора.',
 'В зимнем царстве, где снега белы,  \nДеревня спит под бархатом неба.  \nЛуна, как серебряный ключик,  \nОткрывает тайны ночи 

In [46]:
import pandas as pd 

gen_authors_lst = ['Александр Пушкин']
gen_themes_lst = ['Зима в деревне']

pd.DataFrame({
    'author': gen_authors_lst,
    'theme': gen_themes_lst,
    'queries': gen_queries[0],
    'LLM_LORA_P1': [gen_response_llm_lst_p[0]],
    'LLM_LORA_P2': [gen_response_llm_lst_p[1]],
    'LLM_LORA_P3': [gen_response_llm_lst_p[2]]
}).to_csv('../data/ResponsesGen_llm_loraP.csv', index=False)

In [47]:
lora_name

'pushkin'

In [48]:
unload_lora(lora_name)

True